In [ ]:
from openai import OpenAI
from dotenv import dotenv_values
import json
import math
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

import functions.prompts as prompts

config = dotenv_values(".env")

client = OpenAI(
    api_key=config['DEEPINFRA_TOKEN'],
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
df = pd.read_csv("dump/csv/papers.csv")
# df.iloc[np.r_[0:4, -4:0]]
df.head()

In [ ]:
def gen_body(text, top5=True, model="meta-llama/Llama-3.3-70B-Instruct-Turbo"):
    return {
            "model": model,
            "messages":[{
                    "role": "system",
                    "content": f"{prompts.top5() if top5 else prompts.analysis()}\nPlease respond in valid JSON format that matches this schema: {str(prompts.Top5Model.model_json_schema() if top5 else prompts.AnalysisModel.model_json_schema())}. **IMPORTANT**: ONLY RESPOND WITH AN JSON OBJECT CONTAINING SCORES ACCORDING TO THE ABOVE SCHEMA. THE RESPONSE MUST END WITH A CURLY BRACKET. DO NOT ADD ANALYSIS OR EXPLANATION."
                },
                {
                    "role": "user",
                    "content": text
                }, 
                {
                    "role": "assistant",
                    "content": "{"
                }],
        }

In [ ]:
def llm_paper(client, i, df):
    text = ""
    f = open(f"output/{df.iloc[i]['id']}.txt", "r")
    text = f.read()
    f.close()

    paper = f"PAPER TITLE: {df.iloc[i]['name']}\n\nPAPER TEXT: {text}"
    # gen_body(paper)
    
    model = "google/gemma-3-27b-it"
    
    scores = [ client.chat.completions.create(**gen_body(paper, model=model)) for _ in range(3)]
    analysis = [ client.chat.completions.create(**gen_body(paper, top5=False, model=model)) for _ in range(3)]
    
    return {
        "scores": scores,
        "analysis": analysis
    }
    
def parse_r(r, id, typ):
    # id = r.custom_id
    validateModel = prompts.Top5Model if typ == "top5" else prompts.AnalysisModel
    try:
        # print(r)
        text = r.choices[0].message.content
        if(text.startswith("```json")):
            text = text.split("```json")[1].split("}")[0].replace("'", '"') + "}"
        else:
            text = "{" + text.split("}")[0].replace("'", '"') + "}"
        if(text.startswith("{{")):
            text = text[1:]
        return {
            "id": id,
            "scores": validateModel.model_validate(json.loads(text)).model_dump()
        }
    except Exception as e:
        print(f"Error {e} - {"{" + r.choices[0].message.content}")
        return {
            "id": id,
            "scores": None
        }

def parse_paper(rs):
    scores = [ parse_r(x, j, "top5") for j, x in enumerate(rs['scores']) ]
    analysis = [ parse_r(x, j, "analysis") for j, x in enumerate(rs['analysis']) ]
    return {
        "scores": scores,
        "analysis": analysis
    }

In [ ]:
model_name = "gemma"

def update_df(df, i, no, score, typ):
    metrics = ['score'] if typ == "top5" else ['originality', 'rigor', 'scope', 'impact', 'written_by_ai']
    validateModel = prompts.Top5Model if typ == "top5" else prompts.AnalysisModel

    for _, metric in enumerate(metrics):
        column_name = f"{model_name}-{metric}-{int(no)+1}"
        
        if column_name not in df.columns:
            df[column_name] = None

        try:
            o = validateModel.model_validate(score)
            df.loc[i, column_name] = o.__dict__[metric]
        except:
            print(f"ERROR | Can't update the model in in {column_name}, skipping...")

In [ ]:
def evaluate_paper(client, i, df):
    print(f"Evaluating {i} -> {df.iloc[i]['name']}")
    
    x = llm_paper(client, i, df)
    y = parse_paper(x)
    
    for j, s in enumerate(y['scores']):
        update_df(df, i, j, s['scores'], "top5")
    
    for j, a in enumerate(y['analysis']):
        update_df(df, i, j, a['scores'], "analysis")
    # update_df(df, i, 0, y['scores'][0]['scores'], "top5")
    # idx = df.index[df['id'] == id].tolist()[0]
    # content = l['response']['body']['choices'][0]['message']['content']
    
# evaluate_paper(client, 0, df)    

In [ ]:
import math
from concurrent.futures import ThreadPoolExecutor

chunk = 30
for i in range(0, len(df), chunk):
    print(f"PROCESSING CHUNK {(i // chunk) + 1} of {math.ceil(len(df) / chunk)}")
    with ThreadPoolExecutor(max_workers=200) as executor:
        results = list(executor.map(
            evaluate_paper, 
            [client] * chunk,
            [j for j in range(i, min(i+chunk, len(df)))],
            [df] * len(df["file"]),
        ))
        
    df.to_csv(f"{model_name}/{(i // chunk) + 1}.csv", index=False)

In [ ]:
# Fallback
fallback = df[df.isna().any(axis=1)].index

with ThreadPoolExecutor(max_workers=200) as executor:
        results = list(executor.map(
            evaluate_paper, 
            [client] * len(fallback),
            fallback,
            [df] * len(fallback),
        ))
        
df.to_csv(f"{model_name}/fallback.csv", index=False)

In [20]:
df.head()

,id,file,name,journal,authors,affiliations,len-original,len-anond,gemma-score-1,gemma-score-2,...,gemma-originality-2,gemma-rigor-2,gemma-scope-2,gemma-impact-2,gemma-written_by_ai-2,gemma-originality-3,gemma-rigor-3,gemma-scope-3,gemma-impact-3,gemma-written_by_ai-3
0,1_0,1. Econometrica/ecta200736.pdf,The Political Economy of Zero-Sum Thinking,Econometrica,S. Nageeb Ali; Maximilian Mihm; Lucas Siga,"Department of Economics, Pennsylvania State Un...",16496.0,15956.0,3,3,...,7,6,6,5,8,7,8,6,5,8
1,1_1,1. Econometrica/ecta200731.pdf,Social Media and Collective Action in China,Econometrica,Bei Qin; David Strömberg; Yanhui Wu,"Bei Qin: Department of Accountancy, Economics ...",18206.0,17419.0,4,4,...,7,8,6,7,2,7,8,6,6,2
2,1_2,1. Econometrica/ecta200725.pdf,Ambiguous Contracts,Econometrica,Paul Dütting; Michal Feldman; Daniel Peretz; L...,"Google Research; School of Computer Science, T...",11830.0,11448.0,4,4,...,8,9,7,6,2,8,9,7,6,2
3,1_3,1. Econometrica/ecta200741.pdf,PERSUASION MEETS DELEGATION,Econometrica,Anton Kolotilin; Andriy Zapechelnyuk,"Anton Kolotilin: School of Economics, UNSW Bus...",15756.0,15075.0,6,6,...,8,9,7,6,2,8,9,7,6,2
4,1_4,1. Econometrica/Econometrica - 2025 - Berger -...,"Minimum Wages, Efficiency, and Welfare",Econometrica,David Berger; Kyle Herkenhoff; Simon Mongey,"Economics Department, Duke University; Departm...",18607.0,16735.0,4,4,...,7,9,8,6,2,7,8,7,6,2


In [ ]:
df.to_csv(f"result-{model_name}.csv")